# Task 08 - Support Vector Machines (SVM)
## Pattern Recognition and Machine Learning

Copy and import needed files/methods from previous assignment to this directory. 
Adding path to the previous assignment is not sufficient. Upload system
requires your code to be self contained.

In [ ]:
%load_ext autoreload
%autoreload 2
# uncomment following for interactive matplotlib
# %matplotlib notebook

from svm1 import *
import numpy as np

## Train SVM on simple data

In [ ]:
X = np.array([[1, 2, 1, -1, -1, -2], [1, 1, 2, -1, -2, -1]])
y = np.array([1, 1, 1, -1, -1, -1])
C = np.inf;

[w, b, sv_idx]  = my_svm(X, y, C)

print('\n', 'w: ', w, '\n', 'b: ', b, '\n', 'sv_idx:', sv_idx)

In [ ]:
plot_points(X, y)
plt.axis('equal')
plot_boundary(plt.gca(), w, b, X[:, sv_idx])
plt.title('Soft-margin SVM, C = {}'.format(C));



## Toy data

### Load the data

In [ ]:
data = np.load("data_svm1.npz", allow_pickle=True)
X_toy = data["X"]
y_toy = data["y"]

### Train Soft-margin SVM

In [ ]:
C = 1
w, b, sv_idx = my_svm(X_toy, y_toy, C, options={'verb': False})

### Visualize

In [ ]:
plot_points(X_toy, y_toy)
plot_boundary(plt.gca(), w, b, X_toy[:, sv_idx])
plt.title('Soft-margin SVM, C = {}'.format(C))
plt.axis('equal')
plt.savefig('linear_svm.png')



In [ ]:
try:
    from ipywidgets import interact, interactive, fixed
    
    @interact(C=(0.001,10.,0.001))
    def soft_margin(C=1):
        w, b, sv_idx = my_svm(X_toy, y_toy, C, options={'verb': False})
        plt.axis('equal')
        plot_points(X_toy, y_toy)
        plot_boundary(plt.gca(), w, b, X_toy[:, sv_idx])
        plt.title('Soft-margin SVM, C = {}'.format(C));
        
except ImportError:
    print('Optional feature.')

## OCR data
### Load the data

In [ ]:
data = np.load("data_svm1_images.npz", allow_pickle=True)
tst = data["tst"].item()
trn = data["trn"].item()

X_trn, y_trn, norm_trn = compute_measurements_2d(trn)

### Find optimal C
This will probably take around 10 minutes

In [ ]:
np.random.seed(42)
trn_folds, tst_folds = crossval(y_trn.size, num_folds=10)

best_C = None
best_C_err = np.inf
for C in [10, 1, 0.1, 0.01, 0.001]:
    
    C_err = compute_test_error(trn_folds, tst_folds, X_trn, y_trn, C)
    
    if C_err < best_C_err:
        best_C_err = C_err
        best_C = C
    print('C {} -> err {}'.format(C, C_err))

print('answers.txt 4: best C: {}'.format(best_C))


### Train SVM with optimal C

In [ ]:
w, b, sv_idx = my_svm(X_trn, y_trn, best_C, options={'verb': False})

### visualize

In [ ]:
plt.figure(figsize=(8,8))
plot_points(X_trn, y_trn, size=2)
plot_boundary(plt.gca(), w, b, None)
plt.title('Linear SVM train data')
plt.axis('equal')
plt.savefig('ocr_svm_trn.png')



### Classify test data

In [ ]:
X_tst, y_tst, norm_tst = compute_measurements_2d(tst, norm_trn)
classif = classif_linear_svm(X_tst, {'w': w, 'b': b})

In [ ]:
raise NotImplementedError("You have to implement the rest.")
error = None



### And visualize

In [ ]:
plt.figure(figsize=(8,8))
plot_points(X_tst, y_tst)
plot_boundary(plt.gca(), w, b, None)
plt.axis('equal')
plt.title('Linear SVM test data')
plt.savefig('ocr_svm_tst.png')



In [ ]:
labels = classif.copy()
labels[labels == 1] = 0
labels[labels == -1] = 1

plt.figure()
show_classification(tst['images'], labels, 'AC')
plt.savefig('ocr_svm_classif.png')

